In [1]:
from models.pspnet import PSPNet
from datasets.generic_dataset import GenericSegmentationDataset
from utils.transforms import *
from tools.torch.listeners import *
import matplotlib.pyplot as plt
from torchvision.transforms import *
from PIL import Image
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import numpy as np
from utils.losses import AuxiliaryLoss
from tools.torch.trainers import TorchTrainer
from pathlib import Path

In [2]:
transforms = Transforms([
    SyncResize(size=(473, 473)),
    SyncRandomHorizontalFlip(),
    SyncRandomRotation(degrees=(-10, 10)),
    SyncRandomScaledCrop(scale=(0.5, 2.0)),
    Transform(data=RandomGaussianBlur()),
    Transform(data=ToTensor(), target=lambda x: torch.from_numpy(np.array(x))),
    Transform(data=Normalize(mean=[0.435, 0.382, 0.353], std=[0.290, 0.273, 0.273])),
])

In [3]:
root = r'C:\Users\imana\Desktop\lip'

train_data = GenericSegmentationDataset(root=root, mode='train', transform=transforms)
val_data = GenericSegmentationDataset(root=root, mode='val', transform=transforms)

In [4]:
test_dir = Path('test')
checkpoint_dir = test_dir.joinpath('checkpoint')
log_dir = test_dir.joinpath('log')

In [5]:
listener = Listeners([
    TensorBoardLossReporter(log_dir=log_dir),
    TensorBoardModelReporter(log_dir=log_dir),
])

In [6]:
model = PSPNet(classes=20)

criterion = AuxiliaryLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = TorchTrainer(
    name='pspnet',
    epochs=1,
    device=torch.device('cpu'),
    batch_size=2,
    train_data=train_data,
    val_data=val_data,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    listener=listener,
    checkpoint_dir=checkpoint_dir,
)

trainer.start()

epoch: 1, step: 109, loss: 2.900:   1%|█                                                                                                                                                        | 109/15231 [1:11:22<165:03:04, 39.29s/it]


KeyboardInterrupt: 